<a href="https://colab.research.google.com/github/kmayutrisna/SA_IMDB/blob/main/Glove_CNNLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary library
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, GRU, Bidirectional, concatenate, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Get the dataset from gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Display top of data
data =pd.read_csv("drive/My Drive/data/IMDB Dataset.csv")

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Text preprocessing
def preprocess_text(text):

    # Text cleaning
    text = re.sub('[^a-zA-Z]', ' ', text)

    #Single character removal
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)

    #Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)

    # Lowercasing
    text = text.lower()

    # Lowercasing
    text = text.split()

    # Stopword removal
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if not word in stop_words]

    # Join the tokens back to a single string
    text = ' '.join(text)
    return text

In [ ]:
# Preprocess the text column
data['review'] = data['review'].apply(preprocess_text)

In [ ]:
# Convert sentiment to binary labels
data['sentiment'] = data['sentiment'].map({'positive': 1, 'negative': 0})
#data = data.sample(frac=1).reset_index(drop=True)  # Shuffling the dataset

In [ ]:
data.head()

,review,sentiment
0,one reviewers mentioned watching oz episode ho...,1
1,wonderful little production br br filming tech...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake thinks zombie...,0
4,petter mattei love time money visually stunnin...,1


In [ ]:
X = data['review'].values
y = data['sentiment'].values

In [ ]:
# Splitting the dataset

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.111, random_state=42)
#hasilnya 40005

In [ ]:
# Tokenizing the texts
vocab_size = 10000  # Maximum number of words to keep in the vocabulary
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(X_train)

In [ ]:
# Converting text sequences to integer sequences
train_seq = tokenizer.texts_to_sequences(X_train)
val_seq = tokenizer.texts_to_sequences(X_val)
test_seq = tokenizer.texts_to_sequences(X_test)

In [ ]:
# Padding sequences
#max_sequence_length = max([len(seq) for seq in train_seq])
max_length = 100  # Maximum sequence length
train_data = pad_sequences(train_seq, maxlen=max_length)
val_data = pad_sequences(val_seq, maxlen=max_length)
test_data = pad_sequences(test_seq, maxlen=max_length)

In [ ]:
# Loading the GloVe word embeddings
glove_path = 'drive/My Drive/data/glove.6B.100d.txt'

In [ ]:
import numpy as np
embedding_dim = 100  # Dimensionality of the word embeddings
embedding_index = {} #empty dictionary that will store the word embeddings. Each word will be associated with its corresponding embedding coefficients.
with open(glove_path, 'r') as f:
    for line in f:
        values = line.split() #Inside the loop, the current line is split into individual values using the split method, which splits the line based on whitespace. The resulting values are stored in the values list.
        word = values[0] #The first value in values (index 0) is assigned to the variable word. This represents the word itself.
        coefficients = np.asarray(values[1:], dtype='float32') #The word (word) and its corresponding coefficients (coefficients) are stored as a key-value pair in the embedding_index dictionary.
        embedding_index[word] = coefficients #After the loop ends, the embedding_index dictionary will contain the word embeddings for all words in the file.

In [ ]:
# Creating an embedding matrix
word_index = tokenizer.word_index
num_words = min(vocab_size, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i < vocab_size:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
#Standar biLSTM
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, MaxPooling1D
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Embedding(num_words, embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(train_data, y_train, batch_size=32, epochs=10, validation_data=(val_data, y_val))

Epoch 1/10
1251/1251 [==============================] - 102s 78ms/step - loss: 0.4189 - accuracy: 0.8022 - val_loss: 0.3508 - val_accuracy: 0.8436
Epoch 2/10
1251/1251 [==============================] - 96s 77ms/step - loss: 0.3198 - accuracy: 0.8621 - val_loss: 0.3223 - val_accuracy: 0.8653
Epoch 3/10
1251/1251 [==============================] - 99s 79ms/step - loss: 0.2667 - accuracy: 0.8880 - val_loss: 0.3428 - val_accuracy: 0.8515
Epoch 4/10
1251/1251 [==============================] - 94s 75ms/step - loss: 0.2115 - accuracy: 0.9143 - val_loss: 0.3993 - val_accuracy: 0.8505
Epoch 5/10
1251/1251 [==============================] - 98s 78ms/step - loss: 0.1517 - accuracy: 0.9419 - val_loss: 0.3963 - val_accuracy: 0.8543
Epoch 6/10
1251/1251 [==============================] - 97s 78ms/step - loss: 0.1058 - accuracy: 0.9603 - val_loss: 0.4638 - val_accuracy: 0.8484
Epoch 7/10
1251/1251 [==============================] - 97s 77ms/step - loss: 0.0749 - accuracy: 0.9716 - val_loss: 0.5666 

In [ ]:
# Evaluating the model
y_pred = model.predict(test_data)
y_pred_binary = np.round(y_pred).flatten()

157/157 [==============================] - 5s 28ms/step


In [ ]:
accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.8448
Precision: 0.8623700623700624
Recall: 0.8233425962683605
F1-score: 0.842404549147035
